# Data Preperation

### Importiere Bibliotheken

In [26]:
import pandas as pd
from imblearn.combine import SMOTETomek
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

### Datensatz einlesen
Da Pandas die Datentypen der Merkmale nicht optimal erkennt, werden im Folgenden die konkreten Datentypen definiert. \
Diese können aus der Beschreibung des Datensatztes abgeleitet werden.

In [27]:
dtypes = {
    'UDI': 'int32',
    'Product ID': 'str',
    'Type': 'category',
    'Air temperature [K]': 'float32',
    'Process temperature [K]': 'float32',
    'Rotational speed [rpm]': 'float32',
    'Torque [Nm]': 'float32',
    'Tool wear [min]': 'float32',
    'Machine failure': 'bool',
    'TWF': 'bool',
    'HDF': 'bool',
    'PWF': 'bool',
    'OSF': 'bool',
    'RNF': 'bool'
}

df = pd.read_csv('./dataset.csv', dtype=dtypes)

### Erstelle Dummy-Merkmale für das Merkmal Maschinen-Typ

In [28]:
dummy_columns = pd.get_dummies(df['Type'], prefix='Type')
df = pd.concat([df, dummy_columns], axis=1)

### Definiere die Input- und Output- Merkmale

In [29]:
input_columns = ['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]', 'Type_H', 'Type_L', 'Type_M']
output_column = ['label']
numeric_columns = ['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]']

### Generiere Labeling

In [30]:
def get_label(row):
    for defect in ['TWF', 'HDF', 'PWF', 'OSF', 'RNF']:
        if row[defect] == 1:
            return defect
    return 'no_failure'

df['label'] = df.apply(get_label, axis=1)

### Initialisiere Training- und Testdaten

In [31]:
X = df[input_columns]
y = df[output_column]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

### Implementiere Oversampling auf den Trainingsdaten
Oversampling wird gewählt, da der Datensatz zwei sehr starke Unausgeglichenheiten enthält und ein Undersampling zu einem sehr starken Datenverlust führen würde.\
Dies liegt daran, da nur wenige Machinen tatsächlich einen Defekt aufweisen.

In [32]:
smote = SMOTETomek(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

### Kombiniere Test- und Trainingsdatensatz

In [33]:
df_train_resampled = pd.concat([X_train_resampled, y_train_resampled], axis=1)
df_test = pd.concat([X_test, y_test], axis=1)

### Skaliere numerische Werte mit der Standartskalierung auf Basis der Trainingsdaten

In [34]:
scaler = StandardScaler()
df_train_resampled[numeric_columns] = scaler.fit_transform(df_train_resampled[numeric_columns])
df_test[numeric_columns] = scaler.transform(df_test[numeric_columns])

### Speichere den Trainings- und Testdatensatz

In [35]:
df_train_resampled.to_csv('./dataset_train_resampled.csv')
df_test.to_csv('./dataset_test.csv')

In [36]:
pd.concat([df_train_resampled, df_test]).describe(include='all')

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Type_H,Type_L,Type_M,label
count,48260.000000,48260.000000,48260.000000,48260.000000,48260.000000,48260,48260,48260,48260
unique,NaN,NaN,NaN,NaN,NaN,2,2,2,6
top,NaN,NaN,NaN,NaN,NaN,False,True,False,no_failure
freq,NaN,NaN,NaN,NaN,NaN,40453,40332,31854,9616
mean,-0.011412,-0.006209,0.003261,-0.018562,-0.022014,NaN,NaN,NaN,NaN
std,1.006170,1.009707,0.985318,0.992424,1.002724,NaN,NaN,NaN,NaN
min,-2.884590,-3.655904,-1.068038,-2.858439,-2.090615,NaN,NaN,NaN,NaN
25%,-0.816651,-0.709655,-0.495216,-0.610130,-0.860219,NaN,NaN,NaN,NaN
50%,0.027434,0.095607,-0.328031,0.119388,0.230072,NaN,NaN,NaN,NaN
75%,0.876438,0.692638,0.057681,0.694001,0.902767,NaN,NaN,NaN,NaN
